In [3]:
import os
import re     # this package is needed to remove the special characters in the book
from math import *
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [14]:
working_path = "/Users/yongzhongxu/Downloads/news_article/"

nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

def getFrequancyTable(file):
    word_count = {}
    freq_table = {}
    total = 0
    with open(file, encoding='utf8') as fp:
        for line in fp:
            line = line.rstrip('\n').lower()
            # remove special characters
            line = re.sub('[!@#$._()*%&/]', '', line)
            l=line.split()
            filtered = [w for w in l if not w in stop_words] 
            for word in filtered:
                if word in word_count:
                    word_count[word] += 1
                else:
                    word_count[word] = 1
                total += 1
    fp.close()
    for word in word_count:
        freq_table[word]=word_count[word]/total
        
    return word_count, freq_table, total

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yongzhongxu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
cat1 = "cyb.txt"
cat2 = "mmc.txt"
file1 = working_path+cat1
file2 = working_path+cat2

voc1, table1, total1 = getFrequancyTable(file1)
voc2, table2, total2 = getFrequancyTable(file2)

alpha = 0.0001

#### find the vocabulary table ######
voc = {**voc1, **voc2}
V = len(voc)

### calculate the frequency table using Laplace smoothing
for word in table1:
    table1[word]= (table1[word]+ alpha)/(total1 +V+1)
for word in table2:
    table2[word]= (table2[word] + alpha)/(total2+V+1)

for word in voc:
    a1 = alpha/(total1 +V+1)
    a2 = alpha/(total2 +V+1)
    if word in table1.keys() and word in table2.keys():
        voc[word]= table1[word], table2[word]
    elif (not word in table1.keys() and word in table2.keys()):
        voc[word]= a1, table2[word]
    elif(not word in table2.keys() and word in table1.keys()):
        voc[word]= table1[word], a2
    else:
        voc[word] = a1, a2

voc["security"]

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 315: invalid start byte

In [ ]:
## calculate the conditional probility of each author
def calProb(sample):
    '''
    Returns: p1, p2 are the conditional probability of each category respectively
    '''
    voc3, table3, total3 = getFrequancyTable(file3)
    p1 = 0
    p2 = 0
    for word in voc:
        if word in table3.keys():
            p1 += log(1+table3[word])* log(voc3[word][0])
            p2 += log(1+table3[word])* log(voc3[word][1])
    return p1, p2


def predict(sample):
    file3 = working_path + sample
    p1, p2 = calProb(file3)
    return {True: 0, False: 1}[p1>p2] 
   # here 0 means the author is Conan Doyle, while 1 is Jane Austen.
    

sample = "mmc5.txt"
predict(sample)
      